In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
import json

from datetime import date
import datetime
#imports

There are 20 teams in the PL, each plays 38 matches for a total of 760. The first game in the 2022-2023 season is number 18202, and it counts up one by one from there, so the total range will be 18,202 to 18,961. Then we will take only the completed games from there.

In [2]:
table_url = 'https://theathletic.com/football/premier-league/standings/'

In [3]:
res = requests.get(table_url)
soup = BeautifulSoup(res.content, 'lxml')
table_data = soup.find_all('div', {"class": "sc-c8ee3952-0 sc-83a99c6-6 cOGVhH jOibkY"})
#Getting data for number of total matches

In [4]:
result = table_data[::8]
#The "Matches Played" column is every 9th row so that's what we want -- then slice them so they're only the numbers and sum them

In [5]:
result = [str(i) for i in result]
#Turn soup element into string
result = [result[i].split('</')[0] for i in range(len(result))]
#Then split it and take the element with the number
result = [result[i].split('>')[1] for i in range(len(result))]
#Then split it again and take only the number itself
result = [int(i) for i in result]
#Cast the numbers as integers
total_matches = int(sum(result) / 2)
#Sum elements and divide by two for number of matches

In [6]:
ending_match_id = 18202 + 100 + total_matches
#Adding the 100 to deal with postponed matches. As of right now not sure how to better deal with that since they are
#intermittent throughout the range with no pattern -- nothing in the JSON that indicated 'complete' or 'incomplete'

In [7]:
# d = range(18202,18962)
match_range = range(18202,ending_match_id)

In [8]:
urls = []
for i in match_range:
    base_url = f'https://understat.com/match/{i}'
    urls.append(base_url)

In [9]:
all_res_data = []
for i in urls:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
#     positions = soup.find_all("span", {"class": "player-position"})

    all_res_data.append(scripts)
    #Here we can append only the ones that have 'script'

In [10]:
backup_all_res_data = all_res_data
#Adding a backup unless we need it and so we don't need to scrape again

In [11]:
all_res_data = backup_all_res_data

In [12]:
scripts = []
for i in all_res_data:
    if len(i) > 1:
        scripts.append(i)
    else:
        pass
#All of the incomplete games in all_res_data will have a length of zero, so this is how we will filter out the unplayed games
#if necessary

In [13]:
script_list = []
for i in scripts:
    script_list.append(i[1])
#The [1] element is the one with all of the shot data

In [14]:
for i in range(0, len(script_list)):
    script_list[i] = str(script_list[i])
#turning all of our data into strings

In [15]:
data_list = []

# strip unnecessary symbols and get only JSON data 
for i in range(0, len(script_list)):
    ind_start = script_list[i].index("('")+2 
    ind_end = script_list[i].index("')") 
    json_data = script_list[i][ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
    data = json.loads(json_data)
    data_list.append(data)

In [16]:
all_data = []
match_id_list = []
x_list = []
y_list = []
home_score_list = []
away_score_list = []
player_list = []
xG_list = []
h_a_list = []
situation_list = []
result_list = []
team_list = []
opposing_team_list = []
minute_list = []
length = len(data_list)
for i in range(length):
    match_id = [] 
    x = []
    y = []
    home_score = []
    away_score = []
    player = []
    xG = []
    h_a = []
    situation = []
    result = []
    team = []
    opposing_team = []
    minute = []
    data_away = data_list[i]['a']
    data_home = data_list[i]['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'a_team':
                opposing_team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'situation':
                situation.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'h_goals':
                home_score.append(data_home[index][key])
            if key == 'a_goals':
                away_score.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key]) 
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'h_a':
                h_a.append(data_home[index][key])
        
        #NEED TO APPEND EVERYTHING TO ALL_DATA

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'h_team':
                opposing_team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'situation':
                situation.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'h_goals':
                home_score.append(data_away[index][key])
            if key == 'a_goals':
                away_score.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'h_a':
                h_a.append(data_away[index][key]) 
                
    match_id_list.append(match_id)
    x_list.append(x)
    y_list.append(y)
    team_list.append(team)
    opposing_team_list.append(opposing_team)
    xG_list.append(xG)
    situation_list.append(situation)
    result_list.append(result)
    home_score_list.append(home_score)
    away_score_list.append(away_score)
    minute_list.append(minute)
    player_list.append(player)
    h_a_list.append(h_a)

In [17]:
match_id_list = [val for sublist in match_id_list for val in sublist]
x_list = [val for sublist in x_list for val in sublist]
y_list = [val for sublist in y_list for val in sublist]
team_list = [val for sublist in team_list for val in sublist]
opposing_team_list = [val for sublist in opposing_team_list for val in sublist]
xG_list = [val for sublist in xG_list for val in sublist]
situation_list = [val for sublist in situation_list for val in sublist]
result_list = [val for sublist in result_list for val in sublist]
home_score_list = [val for sublist in home_score_list for val in sublist]
away_score_list = [val for sublist in away_score_list for val in sublist]
minute_list = [val for sublist in minute_list for val in sublist]
player_list = [val for sublist in player_list for val in sublist]
h_a_list = [val for sublist in h_a_list for val in sublist]
#Flattening nested lists

In [18]:
col_names = ['match_id','x','y','xG','minute','situation','result','team','opposing_team','h_a','player','home_score','away_score']
df = pd.DataFrame([match_id_list,x_list,y_list,xG_list,minute_list,situation_list,result_list,team_list,opposing_team_list,h_a_list,player_list,home_score_list,away_score_list],index=col_names)
df = df.T
df = df.astype({'minute':'int'})
df = df.sort_values(by=['match_id','minute'])
df = df.reset_index()

In [19]:
df.isna().sum()
#Checking for null values

index            0
match_id         0
x                0
y                0
xG               0
minute           0
situation        0
result           0
team             0
opposing_team    0
h_a              0
player           0
home_score       0
away_score       0
dtype: int64

# xG

In [20]:
# test_home_list = np.where((df['result'] == 'Goal') & (df['h_a'] == 'h') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'a'), 1, 0)

In [21]:
df['xG'] = df['xG'].astype(float)
df['match_id'] = df['match_id'].astype(int)

In [22]:
xg_total = df.groupby('team')['xG'].sum()
#Count total xG for every team

In [23]:
xg_total = pd.DataFrame(data=xg_total)
# xg_total = xg_total.reset_index(level=0)

In [24]:
xg_total['xGA'] = df.groupby('opposing_team')['xG'].sum()

In [25]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [26]:
xg_total['matches_played'] = df.groupby('team')['match_id'].nunique()
#Get number of matches played for each team

In [27]:
xg_total = xg_total.reset_index(level=0)

In [28]:
xg_total['avg_xG_diff'] = round(xg_total['xG_difference'] / xg_total['matches_played'], 2)
xg_total['avg_xG_for'] = round(xg_total['xG'] / xg_total['matches_played'], 2)
xg_total['avg_xGA'] = round(xg_total['xGA'] / xg_total['matches_played'], 2)

In [29]:
xg_total.isna().sum()
#Checking for null values

team              0
xG                0
xGA               0
xG_difference     0
matches_played    0
avg_xG_diff       0
avg_xG_for        0
avg_xGA           0
dtype: int64

# Non Pen xG

In [30]:
non_pen_xg = df.groupby('team')
non_pen_xg = non_pen_xg.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())
#Getting non pen xG and adding to df
#Not working because this groupby result does not have a name so can't go into a df

In [31]:
non_pen_xga = df.groupby('opposing_team')
non_pen_xga = non_pen_xga.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())

In [33]:
xg_total['non_pen_xG'] = non_pen_xg.values
xg_total['non_pen_xGA'] = non_pen_xga.values

#Now need to match for team

In [34]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [35]:
xg_total['non_pen_xg_difference'] = xg_total['non_pen_xG'] - xg_total['non_pen_xGA']

In [36]:
xg_total['avg_non_pen_xG_diff'] = round(xg_total['non_pen_xg_difference'] / xg_total['matches_played'], 2)
xg_total['avg_non_pen_xG_for'] = round(xg_total['non_pen_xG'] / xg_total['matches_played'], 2)
xg_total['avg_non_pen_xGA'] = round(xg_total['non_pen_xGA'] / xg_total['matches_played'], 2)

In [37]:
xg_total = xg_total.sort_values(by='avg_xG_diff', ascending=False).reset_index(drop=True)

In [38]:
xg_total

,team,xG,xGA,xG_difference,matches_played,avg_xG_diff,avg_xG_for,avg_xGA,non_pen_xG,non_pen_xGA,non_pen_xg_difference,avg_non_pen_xG_diff,avg_non_pen_xG_for,avg_non_pen_xGA
0,Arsenal,29.994216,11.822169,18.172048,14,1.30,2.14,0.84,29.233047,10.299831,18.933217,1.35,2.09,0.74
1,Manchester City,30.873266,13.227611,17.645655,14,1.26,2.21,0.94,28.589760,11.705274,16.884486,1.21,2.04,0.84
2,Newcastle United,29.188017,17.465238,11.722779,15,0.78,1.95,1.16,27.665679,16.704069,10.961610,0.73,1.84,1.11
3,Brighton,25.123187,16.173447,8.949740,14,0.64,1.79,1.16,22.839680,13.128772,9.710909,0.69,1.63,0.94
4,Tottenham,26.476696,18.074510,8.402187,15,0.56,1.77,1.20,24.193190,17.313341,6.879849,0.46,1.61,1.15
5,Liverpool,29.726785,23.566784,6.160001,14,0.44,2.12,1.68,29.726785,21.283278,8.443507,0.60,2.12,1.52
6,Manchester United,22.799225,17.433604,5.365621,14,0.38,1.63,1.25,22.038056,16.672435,5.365621,0.38,1.57,1.19
7,West Ham,19.924927,17.650329,2.274598,15,0.15,1.33,1.18,16.880251,15.366822,1.513429,0.10,1.13,1.02
8,Brentford,23.726381,23.570440,0.155941,15,0.01,1.58,1.57,20.681706,22.809271,-2.127566,-0.14,1.38,1.52
9,Leeds,21.755443,22.389467,-0.634025,14,-0.05,1.55,1.60,20.233105,21.628298,-1.395193,-0.10,1.45,1.54


In [39]:
xg_total.to_excel("prem_xG_data.xlsx")

# In Home Games

# In Away Games

NEXT: Will add something to get all of the players from the match, and then count the defensive players to determine back 4 or back 5

Code will be -- take all of the elements where the class "player-position" is DC... get a count, if it's two then the team will be in a back four, if there are 3 then it's a back five with wingbacks